In [3]:
import pyodbc
import os
import sys
import pandas as pd  # Import pandas for DataFrame handling
import config  # Import config module for database connection details

def connect_to_database():
    try:
        # Get the current script path
        current_path = os.path.dirname(os.path.abspath(__file__))
        print("Current path:", current_path)
    except NameError:
        # Fallback if __file__ is not defined (e.g., in Jupyter)
        current_path = os.getcwd()
        print("Current path:", current_path)

    # Add the parent directory to the system path
    sys.path.append(os.path.dirname(current_path))

    # Database connection parameters from config
    server = config.DefaultConnection['server']
    database = config.DefaultConnection['database']
    username = config.DefaultConnection['username']
    password = config.DefaultConnection['password']

    # Establish database connection
    cnxn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
    return cnxn

def fetch_crime_stats(provincecode, policestation, year=None, quarter=None):
    # Connect to the database
    cnxn = connect_to_database()
    cursor = cnxn.cursor()

    try:
        # Execute the stored procedure with the parameters
        if year:  # Check if the year parameter is not empty
            cursor.execute("EXEC sp_GetQuartilyCrimeStatsPerQuarter ?, ?, ?, ?", (provincecode.strip(), policestation.strip(), int(year), int(quarter)))
        else:
            cursor.execute("EXEC sp_GetQuartilyCrimeStatsPerQuarter ?, ?, ?, ?", (provincecode.strip(), policestation.strip(), None, int(quarter)))

        # Fetch the results after executing the stored procedure
        rows = cursor.fetchall()

        # Fetch the column descriptions (headings)
        headings = [column[0] for column in cursor.description]

        # Reshape the rows data to match the expected shape
        rows = [list(row) for row in rows]

        # Create a DataFrame from the fetched rows and headings
        df = pd.DataFrame(rows, columns=headings)

        # Replace null values with 0's
        df.fillna(0, inplace=True)

        return df  # Return the DataFrame

    except pyodbc.Error as e:
        # Print an error message if there's an exception
        print("Error executing SQL query:", e)
        return None  # Return None if there's an error

    finally:
        # Close the cursor and connection
        cursor.close()
        cnxn.close()

def fetch_all_provinces():
    # Connect to the database
    cnxn = connect_to_database()
    cursor = cnxn.cursor()

    try:
            
        cursor.execute("EXEC sp_GetAllProvinces")

        # Fetch the results after executing the stored procedure
        rows = cursor.fetchall()

        # Fetch the column descriptions (headings)
        headings = [column[0] for column in cursor.description]

        # Reshape the rows data to match the expected shape
        rows = [list(row) for row in rows]

        # Create a DataFrame from the fetched rows and headings
        df = pd.DataFrame(rows, columns=headings)

        # Replace null values with 0's
        df.fillna(0, inplace=True)

        return df  # Return the DataFrame

    except pyodbc.Error as e:
        # Print an error message if there's an exception
        print("Error executing SQL query:", e)
        return None  # Return None if there's an error

    finally:
        # Close the cursor and connection
        cursor.close()
        cnxn.close()

def fetch_policestation_per_provinces(provincecode):
    # Connect to the database
    cnxn = connect_to_database()
    cursor = cnxn.cursor()

    try:
        cursor.execute("EXEC sp_GetPoliceStationsPerProvince ?", (provincecode.strip()))

        # Fetch the results after executing the stored procedure
        rows = cursor.fetchall()

        # Fetch the column descriptions (headings)
        headings = [column[0] for column in cursor.description]

        # Reshape the rows data to match the expected shape
        rows = [list(row) for row in rows]

        # Create a DataFrame from the fetched rows and headings
        df = pd.DataFrame(rows, columns=headings)

        # Replace null values with 0's
        df.fillna(0, inplace=True)

        return df  # Return the DataFrame

    except pyodbc.Error as e:
        # Print an error message if there's an exception
        print("Error executing SQL query:", e)
        return None  # Return None if there's an error

    finally:
        # Close the cursor and connection
        cursor.close()
        cnxn.close()    



ModuleNotFoundError: No module named 'config'

In [1]:
total = 200
print(total)
#print('total = {total}')

200


# Sample prediction dataset

In [ ]:
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import pandas as pd

# Function to train and predict per scenario and algorithm
def train_and_predict(df, models, params, scenario_func, scenario_name, original_df):
    df_encoded = scenario_func(df.copy())
    
    features = [col for col in df_encoded.columns if col not in ['CrimeCategory', 'ProvinceCode', 'PoliceStationCode', 'Quarter', '2023']]
    
    # Split the dataset into features and target variable
    X = df_encoded[features]
    y = df_encoded['TrueValue']  # Adjust this to your target column name

    # Split into train (60%), validation (20%), test (20%)
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    # Scaling: Fit only on train data, transform on train, validation, and test
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    

    # Define K-Fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    realtime_predictions = []
    realtime_metrics = []
    validation_metrics = []

    for name, model in models.items():
        # Perform Grid Search with cross-validation
        grid_search = GridSearchCV(model, params[name], cv=kf, n_jobs=-1, scoring='neg_mean_squared_error')
        grid_search.fit(X_train_scaled, y_train)

        best_model = grid_search.best_estimator_

        # Predictions on validation set
        y_val_pred = best_model.predict(X_val_scaled)

        # Predictions on test set
        y_test_pred = best_model.predict(X_test_scaled)

        # Store validation metrics
        val_metrics = {
            'Algorithm': name,
            'Scenario': scenario_name,
            'Dataset': 'Validation',
            'MAE': mean_absolute_error(y_val, y_val_pred),
            'MSE': mean_squared_error(y_val, y_val_pred),
            'R²': r2_score(y_val, y_val_pred),
            'MAPE': mean_absolute_percentage_error(y_val, y_val_pred)
        }
        validation_metrics.append(pd.DataFrame([val_metrics]))

        # Store test set predictions
        scenario_predictions = pd.DataFrame({
            'CrimeCategory': original_df.loc[y_test.index, 'CrimeCategory'].values,
            'ProvinceCode': original_df.loc[y_test.index, 'ProvinceCode'].values,
            'PoliceStationCode': original_df.loc[y_test.index, 'PoliceStationCode'].values,
            'Quarter': original_df.loc[y_test.index, 'Quarter'].values,
            'Algorithm': [name] * len(y_test),
            'Scenario': [scenario_name] * len(y_test),
            'Prediction': y_test_pred,
            'True_value': y_test
        })

        # Store test set metrics
        test_metrics = {
            'Algorithm': name,
            'Scenario': scenario_name,
            'Dataset': 'Test',
            'MAE': mean_absolute_error(y_test, y_test_pred),
            'MSE': mean_squared_error(y_test, y_test_pred),
            'R²': r2_score(y_test, y_test_pred),
            'MAPE': mean_absolute_percentage_error(y_test, y_test_pred)
        }
        realtime_metrics.append(pd.DataFrame([test_metrics]))

        # Append results
        realtime_predictions.append(scenario_predictions)

    # Concatenate all predictions and metrics after the loop ends
    predictions_df = pd.concat(realtime_predictions, axis=0).reset_index(drop=True)
    test_metrics_df = pd.concat(realtime_metrics, axis=0).reset_index(drop=True)
    val_metrics_df = pd.concat(validation_metrics, axis=0).reset_index(drop=True)

    return predictions_df, test_metrics_df, val_metrics_df
